https://www.kaggle.com/clair14/lgbm-bayesianoptimization

In [ ]:
neg_log_loss

In [61]:
import warnings
warnings.filterwarnings('ignore')
import glob
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from lightgbm import LGBMClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import OneHotEncoder
import random
import os
from sklearn.metrics import log_loss

d = "C:\kaggle_data\credit_card"
lst = os.listdir(d)
print(lst)
train = pd.read_csv(d + '\\' +lst[3])
test = pd.read_csv(d + '\\' +lst[2])
ss = pd.read_csv(d + '\\' +lst[1])
train = train.drop(['index'], axis=1)
train.fillna('NAN', inplace=True) 
test = test.drop(['index'], axis=1)
test.fillna('NAN', inplace=True)

# Married, Civil marriage
train['income_per_size'] = np.log(train['income_total']/train['family_size'])
test['income_per_size'] = np.log(test['income_total']/test['family_size'])
train.loc[(train['family_type']=='Married')|(train['family_type']=='Civil marriage'),'income_per_size']\
= train['income_per_size'] * 2

test.loc[(test['family_type']=='Married')|(test['family_type']=='Civil marriage'),'income_per_size']\
= test['income_per_size'] * 2

def simple_marry(x):
    if x == 'Married' or x =='Civil marriage':
        return '0'
    elif x == 'Separated' or x == 'Widow':
        return '1'
    else:
        return '2'

for df in [train,test]:
    df['family_bins'] = df['family_type'].apply(simple_marry)

# income_total을 로그변환
train['income_total'] = np.log(train['income_total'])
# train = train.drop('income_total',1)
test['income_total'] = np.log(test['income_total'])
# test = test.drop('income_total',1)

# car와 reality를 합친 새로운 칼럼 careality
train['car'] =train['car'].apply(lambda x: int(x=='Y'))
train['reality'] =train['reality'].apply(lambda x: int(x=='Y'))
test['car'] =test['car'].apply(lambda x: int(x=='Y'))
test['reality'] =test['reality'].apply(lambda x: int(x=='Y'))

train['careality'] = train['car'] + train['reality']
train = train.drop(['car', 'reality'],1)

test['careality'] = test['car'] + test['reality']
test = test.drop(['car', 'reality'],1)

object_col = []
for col in train.columns:
    if train[col].dtype == 'object':
        object_col.append(col)

enc = OneHotEncoder()
enc.fit(train.loc[:,object_col])


train_onehot_df = pd.DataFrame(enc.transform(train.loc[:,object_col]).toarray(), 
             columns=enc.get_feature_names(object_col))
train.drop(object_col, axis=1, inplace=True)
train = pd.concat([train, train_onehot_df], axis=1)

test_onehot_df = pd.DataFrame(enc.transform(test.loc[:,object_col]).toarray(), 
             columns=enc.get_feature_names(object_col))
test.drop(object_col, axis=1, inplace=True)
test = pd.concat([test, test_onehot_df], axis=1)

## 제곱
for df in [train,test]:
    df['income_per_size'] = df['income_per_size'].apply(lambda x: x**2)
    
c = 'income_per_size'
mean = train[c].mean()
std = train[c].std()

# 기존 
train[c] = (train[c]-mean)/std
test[c] = (test[c]-mean)/std

#####################
c = 'income_total'
k = 2.2
mean = train[c].mean()
std = train[c].std()
idxs = train.loc[(train[c]>= mean + k*std)|\
                (train[c]<= mean - k*std)].index
train = train.drop(idxs).reset_index(drop=True)

# 기존 
train[c] = (train[c]-mean)/std
test[c] = (test[c]-mean)/std

#####################
c = 'begin_month'
k = 2.2

train[c] = train[c].apply(lambda x: x**2)
test[c] = test[c].apply(lambda x: x**2)

mean = train[c].mean()
std = train[c].std()

idxs = train.loc[(train[c]>= mean + k*std)|\
                (train[c]<= mean - k*std)].index
# train = train.drop(idxs).reset_index(drop=True)

# 기존 
train[c] = (train[c]-mean)/std
test[c] = (test[c]-mean)/std

######################
c = 'DAYS_BIRTH'
k = 2.2
mean = train[c].mean()
std = train[c].std()
idxs = train.loc[(train[c]>= mean + k*std)|\
                (train[c]<= mean - k*std)].index
train = train.drop(idxs).reset_index(drop=True)

#기존 
train[c] = (train[c]-mean)/std
test[c] = (test[c]-mean)/std



##########################
# out: 아웃라이어
out_train = train.loc[train.DAYS_EMPLOYED>0]
out_test = test.loc[test.DAYS_EMPLOYED>0]

in_train = train.loc[train.DAYS_EMPLOYED<=0]
in_test = test.loc[test.DAYS_EMPLOYED<=0]




###################
c = 'DAYS_EMPLOYED'
k = 2.2
mean = in_train[c].mean()
std = in_train[c].std()
idxs = in_train.loc[(in_train[c]>= mean + k*std)|\
                (in_train[c]<= mean - k*std)].index
in_train = in_train.drop(idxs).reset_index(drop=True)

# 기존 
in_train[c] = (in_train[c]-mean)/std
in_test[c] = (in_test[c]-mean)/std

['.ipynb_checkpoints', 'sample_submission.csv', 'test.csv', 'train.csv', 'Untitled.ipynb']


In [71]:
def lgb_eval(num_leaves,max_depth,lambda_l2,lambda_l1,min_child_samples, min_data_in_leaf):
    
    lgtrain = lightgbm.Dataset(train.drop('credit',1), train['credit'])
    params = {
        "objective" : "multiclass",
        "metric" : "multi_logloss", 
        "num_classes" : 3,
        'is_unbalance': True,
        "num_leaves" : int(num_leaves),
        "max_depth" : int(max_depth),
        "lambda_l2" : lambda_l2,
        "lambda_l1" : lambda_l1,
        "num_threads" : 20,
        "min_child_samples" : int(min_child_samples),
        'min_data_in_leaf': int(min_data_in_leaf),
        "learning_rate" : 0.03,
        "subsample_freq" : 5,
        "bagging_seed" : 42,
        "verbosity" : -1
    }
    lgtrain = lightgbm.Dataset(train.drop('credit',1), train['credit'])
    cv_result = lightgbm.cv(params,
                       lgtrain,
                       1000,
                       early_stopping_rounds=100,
                       stratified=True,
                       nfold=10)
    return -cv_result['multi_logloss-mean'][-1]

In [ ]:
lgbBO = BayesianOptimization(lgb_eval, {'num_leaves': (25, 4000),
                                                'max_depth': (5, 63),
                                                'lambda_l2': (0.0, 0.05),
                                                'lambda_l1': (0.0, 0.05),
                                                'min_child_samples': (50, 10000),
                                                'min_data_in_leaf': (100, 2000)
                                                })

lgbBO.maximize(n_iter=10, init_points=2)

|   iter    |  target   | lambda_l1 | lambda_l2 | max_depth | min_ch... | min_da... | num_le... |
-------------------------------------------------------------------------------------------------
|  1        | -0.7695   |  0.01516  |  0.007999 |  45.69    |  2.196e+0 |  962.4    |  3.713e+0 |
|  2        | -0.7438   |  0.0113   |  0.03086  |  13.07    |  2.226e+0 |  339.0    |  1.075e+0 |
|  3        | -0.7716   |  0.02831  |  0.02881  |  18.66    |  1.556e+0 |  1.017e+0 |  3.389e+0 |
|  4        | -0.7351   |  0.01393  |  0.01988  |  32.93    |  3.286e+0 |  326.7    |  62.22    |
|  5        | -0.753    |  0.03567  |  0.000720 |  30.26    |  3.42e+03 |  598.6    |  1.974e+0 |
|  6        | -0.7845   |  0.01822  |  0.01431  |  34.47    |  5.679e+0 |  1.441e+0 |  2.853e+0 |
|  7        | -0.7328   |  0.02989  |  1.736e-0 |  30.76    |  3.314e+0 |  251.8    |  220.2    |
|  8        | -0.7308   |  0.007241 |  0.04318  |  48.2     |  3.314e+0 |  195.3    |  243.4    |


In [62]:
x_train = out_train.drop(['credit'], axis=1)
y_train = out_train['credit']


In [68]:
y_train.isnull().sum()

0

In [64]:
# Logloss 계산하는 함수 (fold=10, statified)

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import log_loss

def XGB_cv(max_depth,learning_rate, n_estimators, gamma
           ,min_child_weight, max_delta_step, subsample
           ,colsample_bytree, silent=True, nthread=-1):
    n = 10
    scores = 0
    kfold = StratifiedKFold(n_splits=n, shuffle=True, random_state=42)
    folds=[]
    for train_idx, valid_idx in kfold.split(x_train, y_train):
      folds.append((train_idx, valid_idx))
    models = []
    for fold in range(n):
        train_idx, valid_idx = folds[fold]
        X_train, X_valid, Y_train, Y_valid = x_train.iloc[train_idx].values, x_train.iloc[valid_idx].values,\
                                         y_train[train_idx].values, y_train[valid_idx].values 
        model = XGBClassifier(max_depth=int(max_depth),
                                learning_rate=learning_rate,
                                n_estimators=int(n_estimators),
                                silent=silent,
                                nthread=nthread,
                                gamma=gamma,
                                min_child_weight=min_child_weight,
                                max_delta_step=max_delta_step,
                                subsample=subsample,
                                colsample_bytree=colsample_bytree)
        model.fit(X_train, Y_train)

        Y_pred = model.predict_proba(X_valid)
        nlog_loss = log_loss(Y_valid,Y_pred)
        scores += nlog_loss
    return -scores

# !pip install bayesian-optimization
from bayes_opt import BayesianOptimization

# 베이지안 사용. pbounds는 각 파라미터의 구간으로 설정
pbounds = {'max_depth': (5, 10),
          'learning_rate': (0.01, 0.3),
          'n_estimators': (50, 1000),
          'gamma': (1., 0.01),
          'min_child_weight': (2, 10),
          'max_delta_step': (0, 0.1),
          'subsample': (0.7, 0.8),
          'colsample_bytree' :(0.5, 0.99)
          }


xgboostBO = BayesianOptimization(f = XGB_cv,pbounds = pbounds, verbose = 2, random_state = 1 )

# 메소드를 이용해 최대화!
xgboostBO.maximize(init_points=2, n_iter = 10)

xgboostBO.max # 찾은 파라미터 값 확인






|   iter    |  target   | colsam... |   gamma   | learni... | max_de... | max_depth | min_ch... | n_esti... | subsample |
-------------------------------------------------------------------------------------------------------------------------


KeyError: "Passing list-likes to .loc or [] with any missing labels is no longer supported. The following labels were missing: Int64Index([   0,    1,    2,    3,    4,\n            ...\n            4193, 4195, 4196, 4197, 4198],\n           dtype='int64', length=3194). See https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike"

In [36]:
len(train)

25708

In [37]:
len(in_train)

20535

In [38]:
len(out_train)

4199

In [ ]:
len()

# #######모델링, 학습 시작#######

|365243?|학습|예측|
|----|----|----|
|yes|out_train|out_test|
|no|in_train|in_test|

In [7]:
# !pip install bayesian-optimization

  Created wheel for bayesian-optimization: filename=bayesian_optimization-1.2.0-py3-none-any.whl size=11685 sha256=12969a168a51b80a0b429a37d0a5ee1fce10efd5534eea80716ecb7b2afe6494
  Stored in directory: c:\users\fizz5\appdata\local\pip\cache\wheels\fd\9b\71\f127d694e02eb40bcf18c7ae9613b88a6be4470f57a8528c5b
Successfully built bayesian-optimization


In [8]:
from bayes_opt import BayesianOptimization

In [21]:
train.dtypes

child_num                                   int64
income_total                              float64
DAYS_BIRTH                                float64
DAYS_EMPLOYED                             float64
FLAG_MOBIL                                  int64
work_phone                                  int64
phone                                       int64
email                                       int64
family_size                               float64
begin_month                               float64
credit                                    float64
income_per_size                           float64
careality                                   int64
gender_F                                  float64
gender_M                                  float64
income_type_Commercial associate          float64
income_type_Pensioner                     float64
income_type_State servant                 float64
income_type_Student                       float64
income_type_Working                       float64


In [45]:
out_train['credit'].value_counts()

2.0    2667
1.0    1025
0.0     507
Name: credit, dtype: int64

In [23]:
import lightgbm

In [55]:
np.array(train['credit']).reshape(-1,1)

array([[1.],
       [1.],
       [2.],
       ...,
       [2.],
       [2.],
       [2.]])

In [57]:
# Logloss 계산하는 함수 (fold=10, statified)

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import log_loss

def lgb_eval(num_leaves,max_depth,lambda_l2,lambda_l1,min_child_samples, min_data_in_leaf):
    train = out_train
    params = {
        "objective" : "multiclass",
        "metric" : "multi_logloss", 
        'is_unbalance': True,
        "num_leaves" : int(num_leaves),
        "max_depth" : int(max_depth),
        "lambda_l2" : lambda_l2,
        "lambda_l1" : lambda_l1,
        "num_threads" : 20,
        "min_child_samples" : int(min_child_samples),
        'min_data_in_leaf': int(min_data_in_leaf),
        "learning_rate" : 0.03,
        "subsample_freq" : 5,
        "bagging_seed" : 42,
        "verbosity" : -1,
        "num_class" : 3
    }
    
    
    lgtrain = lightgbm.Dataset(train.drop(['credit'],1), train['credit'])
    cv_result = lightgbm.cv(params,
                   lgtrain,
                   1000,
                   early_stopping_rounds=100,
                   stratified=True,
                   nfold=10)
    return cv_result['multi_logloss-mean']
    
    
# from bayes_opt import BayesianOptimization

lgbBO = BayesianOptimization(lgb_eval, {'num_leaves': (25, 4000),
                                                'max_depth': (5, 63),
                                                'lambda_l2': (0.0, 0.05),
                                                'lambda_l1': (0.0, 0.05),
                                                'min_child_samples': (50, 10000),
                                                'min_data_in_leaf': (100, 2000)
                                                })

lgbBO.maximize(n_iter=10, init_points=2)

lgbBO.max # 찾은 파라미터 값 확인

|   iter    |  target   | lambda_l1 | lambda_l2 | max_depth | min_ch... | min_da... | num_le... |
-------------------------------------------------------------------------------------------------


ValueError: all the input arrays must have same number of dimensions, but the array at index 0 has 1 dimension(s) and the array at index 1 has 2 dimension(s)

In [41]:
def lgb_cv(n_estimators, learning_rate, subsample, colsample_bytree, reg_alpha, reg_lambda, x_data=None, y_data=None, n=10, output='score'):
    score = 0
    kfold = StratifiedKFold(n_splits=n, shuffle=True, random_state=42)
    models = []
    for train_index, valid_index in kf.split(x_data):
        X_train, Y_train = x_data.iloc[train_index], y_data[train_index]
        X_valid, Y_valid = x_data.iloc[valid_index], y_data[valid_index]
        
        
        model = LGBMClassifier(
            n_estimators = int(n_estimators), 
            learning_rate = learning_rate,
            subsample = np.clip(subsample, 0, 1), 
            colsample_bytree = np.clip(colsample_bytree, 0, 1), 
            reg_alpha = reg_alpha, 
            reg_lambda = reg_lambda,
        )
        
        model.fit(x_train, y_train)
        models.append(model)
        
        Y_pred = model.predict_proba(X_valid)
        nlog_loss = log_loss(Y_valid,Y_pred)
    if output == 'score':
        return nlog_loss
    if output == 'model':
        return models



# bayesian optimization
#xgb_sk = XGBClassifier()
params ={'n_estimators' : (1000,8000),
         'max_depth' : (5,80),
         'gamma' : (0,1),
         'learnig_rate':(0.01, 0.05),
         'min_child_weight':(2,5),
         'subsample':(0.4,1),
         'colsample_bytree':(0.4,1),
         'reg_alpha': (1e-5,100)
}
xgb_bayes = BayesianOptimization(lgb_cv(), pbounds = params, random_state=42)
xgb_max = xgb_bayes.maximize(init_points=5, n_iter=20, acq='ei')
print(xgb_max)

TypeError: lgb_cv() missing 6 required positional arguments: 'n_estimators', 'learning_rate', 'subsample', 'colsample_bytree', 'reg_alpha', and 'reg_lambda'

In [59]:
x_train, y_train = out_train.drop('credit',1), out_train['credit']

In [60]:
# Logloss 계산하는 함수 (fold=10, statified)

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import log_loss

def XGB_cv(max_depth,learning_rate, n_estimators, gamma
           ,min_child_weight, max_delta_step, subsample
           ,colsample_bytree, silent=True, nthread=-1):
    
    n = 10
    scores = 0
    kfold = StratifiedKFold(n_splits=n, shuffle=True, random_state=42)
    folds=[]
    for train_idx, valid_idx in kfold.split(x_train, y_train):
      folds.append((train_idx, valid_idx))
    models = []
    for fold in range(n):
      train_idx, valid_idx = folds[fold]
      X_train, X_valid, Y_train, Y_valid = x_train.iloc[train_idx].values, x_train.iloc[valid_idx].values,\
                                         y_train[train_idx].values, y_train[valid_idx].values 
      model = XGBClassifier(max_depth=int(max_depth),
                                learning_rate=learning_rate,
                                n_estimators=int(n_estimators),
                                silent=silent,
                                nthread=nthread,
                                gamma=gamma,
                                min_child_weight=min_child_weight,
                                max_delta_step=max_delta_step,
                                subsample=subsample,
                                colsample_bytree=colsample_bytree)
      model.fit(X_train, Y_train)
        
      Y_pred = model.predict_proba(X_valid)
      nlog_loss = log_loss(Y_valid,Y_pred)
      scores += nlog_loss
    return scores

!pip install bayesian-optimization
from bayes_opt import BayesianOptimization

# 베이지안 사용. pbounds는 각 파라미터의 구간으로 설정
pbounds = {'max_depth': (5, 10),
          'learning_rate': (0.01, 0.3),
          'n_estimators': (50, 1000),
          'gamma': (1., 0.01),
          'min_child_weight': (2, 10),
          'max_delta_step': (0, 0.1),
          'subsample': (0.7, 0.8),
          'colsample_bytree' :(0.5, 0.99)
          }


xgboostBO = BayesianOptimization(f = XGB_cv,pbounds = pbounds, verbose = 2, random_state = 1 )

# 메소드를 이용해 최대화!
xgboostBO.maximize(init_points=2, n_iter = 10)

xgboostBO.max # 찾은 파라미터 값 확인






|   iter    |  target   | colsam... |   gamma   | learni... | max_de... | max_depth | min_ch... | n_esti... | subsample |
-------------------------------------------------------------------------------------------------------------------------


KeyError: "Passing list-likes to .loc or [] with any missing labels is no longer supported. The following labels were missing: Int64Index([   0,    1,    2,    3,    4,\n            ...\n            4193, 4195, 4196, 4197, 4198],\n           dtype='int64', length=3194). See https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike"

In [ ]:
random.seed(42)
skf = StratifiedKFold(n_splits=n, shuffle=True, random_state=42)

In [14]:

folds=[]
losses=[]
for train_idx, valid_idx in skf.split(train, train['credit']):
    folds.append((train_idx, valid_idx))

    
    
lgb_models={}
for fold in range(n):
    train_idx, valid_idx = folds[fold]
    X_train, X_valid, y_train, y_valid = train.drop(['credit'],axis=1).iloc[train_idx].values, train.drop(['credit'],axis=1).iloc[valid_idx].values,\
                                         train['credit'][train_idx].values, train['credit'][valid_idx].values 
    lgb = LGBMClassifier(n_estimators=1000)
    lgb.fit(X_train, y_train, 
            eval_set=[(X_train, y_train), (X_valid, y_valid)], 
            early_stopping_rounds=30,
           verbose=100)
    lgb_models[fold]=lgb
    
    
    
    losses.append(log_loss(y_valid, lgb.predict_proba(X_valid)))
print(sum(losses)/n)

====================================1============================================
Training until validation scores don't improve for 30 rounds
[100]	training's multi_logloss: 0.617011	valid_1's multi_logloss: 0.74062
[200]	training's multi_logloss: 0.521674	valid_1's multi_logloss: 0.733707
Early stopping, best iteration is:
[211]	training's multi_logloss: 0.51266	valid_1's multi_logloss: 0.732927


====================================2============================================
Training until validation scores don't improve for 30 rounds
[100]	training's multi_logloss: 0.620542	valid_1's multi_logloss: 0.743797
[200]	training's multi_logloss: 0.51959	valid_1's multi_logloss: 0.73226
Early stopping, best iteration is:
[254]	training's multi_logloss: 0.478933	valid_1's multi_logloss: 0.730607


====================================3============================================
Training until validation scores don't improve for 30 rounds
[100]	training's multi_logloss: 0.616593	valid_1's 

# #######모델링, 학습 끝#######

In [ ]:
in_zeros = np.zeros([len(in_test),3])
for fold in range(n):
    in_zeros += in_models[fold].predict_proba(in_test)/n
in_output = pd.DataFrame(in_zeros)
in_output = in_output.reindex(index=pd.Index(idx_in_test))

In [ ]:
out_zeros = np.zeros([len(out_test),3])
for fold in range(n):
    out_zeros += out_models[fold].predict_proba(out_test)/n
out_output = pd.DataFrame(out_zeros)
out_output = out_output.reindex(index=pd.Index(idx_out_test))

In [ ]:
in2 = np.concatenate((in_zeros,np.array(idx_in_test).reshape(-1,1)),axis=1)
out2 = np.concatenate((out_zeros,np.array(idx_out_test).reshape(-1,1)),axis=1)

In [ ]:
output = pd.DataFrame(np.concatenate((in2,out2),axis=0),columns=[0,1,2,'index'])
output['index'] = output['index'].astype('int')

In [ ]:
for i in range(len(output)):
    row = output.loc[output['index']==i]
    ss.iloc[i,1:] = row.iloc[:,:3]

In [ ]:
ss.to_csv('two_models.csv', index=False)